# Upscale GEDI biomass with PALSAR-2 backscatter

This practical will take the GEDI data we used in the last notebook, overlay it with PALSAR data and look for a relationship between the two. First we will load GEDI data into RAM using the same method as the last notebook.

The code for these tasks is held within the ***gediL4Areader.py*** file in the same directory as this notebook.

In [ ]:
# import libraries
import numpy as np
from gediL4Areader import gediL4A,dataTable,palsar

# define a filename
gediName='/Users/dougal/data/teaching/active_sensing/10_fusion/gedi/L4A/small.GEDI04_A_2020173201139_O08642_03_T02179_02_002_02_V002.h5'

# read the data
gedi=gediL4A(gediName)

# filter out poor quality data
gedi.filterQuality()

Now load the PALSAR data and check that it looks sensible. The files are large and this may take a minute.

*Do these figures match those you have in QGIS?*

In [ ]:
# import libraries
import rasterio
import matplotlib.pyplot as plt

# open the two files
palsarHHname='/Users/dougal/data/teaching/active_sensing/10_fusion/palsar/merged_HH.tif'
palsarHVname='/Users/dougal/data/teaching/active_sensing/10_fusion/palsar/merged_HV.tif'
palsarHH=palsar(palsarHHname)
palsarHV=palsar(palsarHVname)


# plot to the screen for sanity check
palsarHH.plotImage(graphTitle='PALSAR-2 HH backscatter')
palsarHV.plotImage(graphTitle='PALSAR-2 HV backscatter')


## Determining the relationship

We have points of AGBD estimates from GEDI and a raster of backscatter from PALSAR-2. Is there a useful relationship between one and the other? To test this we need to make a table of GEDI AGBD values and backscatter from the corresponding PALSAR-2 pixels.

Run the code below to make a plot and determine the linear correlation.

*Which layer has the higher correlation? Why do you think that is, based on your understanding of radars?*

In [ ]:
# make a table from the two datasets
mergedData=dataTable(gedi,palsarHH,palsarHV)

# plot that table up
mergedData.plotHH()
mergedData.plotHV()

# determine linear correlation
mergedData.correlHH()
mergedData.correlHV()

## Machine learning

The relationship is not entirely clear. Machine learning is a useful for finding the best relationship between a variable of interest and multiple variables of interest. Here we will use it to predict AGBD from a combination of PALSAR-2 HH and HV backscatter.

We will use the random forest algoithm as implemented in the sklearn python package. The code below will split the data into a training and a validate set, then train a model to predict GEDI's AGBD from PALSAR with the table of data we have extracted and then predict biomass across all of the PALSAR data.

*Does the scatterplot look accurate?*

In [ ]:
# parameters for our RF
n_estimators=200
max_depth=None

# split into 70% training, 30% validation
mergedData.splitData(trainFrac=0.7)

# calibrate the model
mergedData.buildRF(n_estimators,max_depth)
    
# predict the model
mergedData.predict()

# make a scatterpot of all predicted versus GEDI AGBD
mergedData.scatterAll()

## Validation

Our scatterpot gives us an idea, but we must validate our model with data indepdent from that used to train. The command below will use the 30% of the data we have reserved to calculate some standard error metrics.

*Is this a useful model? What could be done to improve it? Try changing the RF settings and rerunning. What pre-processing could you try on the PALSAR or GEDI data?*

In [ ]:
# validate
mergedData.validateRF()

The properties of the data are likely to influence the accuracy. Things to look out for are saturation, errors as a function of slope (needs a DEM) and errors as a function of signal strength. We have the data to do the first and last here. For signal strength, lidar beam sensitivity is likely to affect the accuracy. Check that by running the code below. 

In [ ]:
# check accuracy as a function of GEDI sensitivity
mergedData.plotError()

## Produce a map

We now have a model that can be applied to any PALSAR-2 data to map biomass. Running the code below will produce a biomass map from the whole PALSAR-2 scene. This may take a while to run.

In [ ]:
mergedData.mapAll()

## Spatial validation

The above randomly selected footprints from within the GEDI flightline. This means that the calibration and validation data overlap in space and time. It is a more stringent test of a model to validate the data at a different space and time to where it was calibrated. That can be done by splitting the training and validation data spatially. The code below splits the data by latitude.

*What does this do to the accuracy results?*

In [ ]:
# parameters for our RF
n_estimators=200
max_depth=None

# split into 70% training, 30% validation
mergedData.splitSpatially(trainFrac=0.7)

# calibrate the model
mergedData.buildRF(n_estimators,max_depth)

# validate
mergedData.validateRF()

## Multiple GEDI files

Up until now we have read a single GEDI file. We could repeat the process with multiple GEDI files.

In [ ]:
# read all GEDI files in a directory
from gediL4Areader import gediL4Amulti

# specify directory and filename root
gediDir='/Users/dougal/data/teaching/active_sensing/10_fusion/gedi/L4A'
gediRoot='small.GEDI04_A'

# read all GEDI data in that directory
gediAll=gediL4Amulti(gediDir,gediRoot)

# filter
gediAll.filterQuality()

We could then repeat the calibration and validation process with this much larger dataset. This will take some time to run.

In [ ]:
# make table of GEDI and PALSAR data
mergedData=dataTable(gediAll,palsarHH,palsarHV)

# parameters for our RF
n_estimators=200
max_depth=None

# split into 70% training, 30% validation
mergedData.splitSpatially(trainFrac=0.7)

# calibrate the model
mergedData.buildRF(n_estimators,max_depth)

# validate
mergedData.validateRF()